In [3]:
from dotenv import load_dotenv
load_dotenv()
import os

import uuid
from typing import List
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.runnables import RunnableConfig

from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.store.memory import InMemoryStore
from langgraph.store.base import BaseStore

# =========================================================
# 1) Long-Term Memory Store
# =========================================================
store = InMemoryStore()

# =========================================================
# 2) System Prompt (for personalized chat)
# =========================================================
SYSTEM_PROMPT = """You are a helpful assistant with memory.

Personalize replies if memory exists.
Always use the user's name if known.
Refer to known preferences or projects if relevant.

In the end, suggest 3 relevant follow-up questions.

USER MEMORY:
{memory}
"""

# =========================================================
# 3) Memory Extraction Model (Structured)
# =========================================================
memory_llm = ChatOpenAI(
    model="xiaomi/mimo-v2-flash:free",
    api_key=os.environ.get('OPEN_ROUTER_API_KEY'),
    base_url="https://openrouter.ai/api/v1",
)

class MemoryItem(BaseModel):
    text: str = Field(description="Atomic long-term memory")
    is_new: bool = Field(description="True if new, false if duplicate")

class MemoryDecision(BaseModel):
    should_write: bool
    memories: List[MemoryItem] = Field(default_factory=list)

memory_extractor = memory_llm.with_structured_output(MemoryDecision)

MEMORY_PROMPT = """You manage long-term user memory.

EXISTING MEMORY:
{existing}

TASK:
- Extract stable user info (name, profession, preferences, long-term projects).
- Mark is_new=true only if not already present.
- Atomic short sentences only.
- No guessing.
- If nothing useful, return should_write=false.
"""

# =========================================================
# 4) Node A: Remember (write-only)
# =========================================================
def remember_node(state: MessagesState, config: RunnableConfig, *, store: BaseStore):
    user_id = config["configurable"]["user_id"]
    ns = ("user", user_id, "profile")

    existing_items = store.search(ns)
    existing = "\n".join(i.value["data"] for i in existing_items) if existing_items else "(empty)"

    last_user_msg = state["messages"][-1].content

    decision: MemoryDecision = memory_extractor.invoke([
        SystemMessage(content=MEMORY_PROMPT.format(existing=existing)),
        HumanMessage(content=last_user_msg)
    ])

    if decision.should_write:
        for mem in decision.memories:
            if mem.is_new:
                store.put(ns, str(uuid.uuid4()), {"data": mem.text})

    return {}

# =========================================================
# 5) Node B: Chat (read-only memory)
# =========================================================
chat_llm = ChatOpenAI(
    model="xiaomi/mimo-v2-flash:free",
    api_key=os.environ.get('OPEN_ROUTER_API_KEY'),
    base_url="https://openrouter.ai/api/v1",
)

def chat_node(state: MessagesState, config: RunnableConfig, *, store: BaseStore):
    user_id = config["configurable"]["user_id"]
    ns = ("user", user_id, "profile")

    items = store.search(ns)
    memory_blob = "\n".join(i.value["data"] for i in items) if items else "(empty)"

    system_msg = SystemMessage(
        content=SYSTEM_PROMPT.format(memory=memory_blob)
    )

    response = chat_llm.invoke([system_msg] + state["messages"])
    return {"messages": [response]}

# =========================================================
# 6) Build Graph
# =========================================================
builder = StateGraph(MessagesState)

builder.add_node("remember", remember_node)
builder.add_node("chat", chat_node)

builder.add_edge(START, "remember")
builder.add_edge("remember", "chat")
builder.add_edge("chat", END)

graph = builder.compile(store=store)

# =========================================================
# 7) Run Demo
# =========================================================
config = {"configurable": {"user_id": "u1"}}

print("\n--- 1 ---")
out = graph.invoke({"messages": [HumanMessage(content="Hi, my name is Nitish")]}, config)
print(out["messages"][-1].content)

print("\n--- 2 ---")
out = graph.invoke({"messages": [HumanMessage(content="I teach AI on YouTube")]}, config)
print(out["messages"][-1].content)

print("\n--- 3 ---")
out = graph.invoke({"messages": [HumanMessage(content="Explain GenAI simply")]}, config)
print(out["messages"][-1].content)

print("\n--- MEMORY ---")
for item in store.search(("user", "u1", "profile")):
    print("-", item.value["data"])



--- 1 ---
Hello Nitish! Nice to meet you. I'm your AI assistant. How can I help you today?

--- 2 ---
Hello! That’s fantastic — teaching AI on YouTube is such an important and growing space. Whether you're explaining foundational concepts, diving into machine learning projects, or exploring the latest in generative AI, you're helping demystify a complex field for a wide audience.

If you’re open to sharing, I’d love to hear more about:
- What topics or formats work best for your viewers?
- Do you focus more on theory, coding tutorials, or applied AI projects?
- Are there any specific challenges you face when creating AI content for YouTube?

I’m here to help with content ideas, script outlines, simplifying complex topics, or even brainstorming engaging ways to present AI concepts. Let me know how I can support your channel! 🎥🤖

Here are a few questions you might find useful to explore next:
1. What’s the most common misconception about AI that you address in your videos?
2. How do you